In [241]:
# Importación de librerias:
import redis
import hashlib
import requests
import json
import geopy.distance
import pymongo
from pymongo import MongoClient
import time
import airportsdata
import pandas as pd
import datetime
import folium
from folium.plugins import MarkerCluster
from folium.plugins import BeautifyIcon

In [7]:
r = redis.Redis(host='127.0.0.1', port=6379, db=0)

In [49]:
client = MongoClient()
db = client.proyecto
vuelos = db.vuelos

In [15]:
# Generamos la función que especifique el estatus del vuelo.

def status(coords_2):
    coords_1 = (19.436111, -99.071944)
    dis =  geopy.distance.geodesic(coords_1, coords_2).km
    if dis < 20:
        return('L')
    else:
        return('A')

In [220]:
def ingresa(air_code, maxflights):
    
    airports = airportsdata.load('IATA')  # key is IATA code
    
    # Conexión a API
    url = 'https://api.flightstats.com/flex/flightstatus/rest/v2/json/airport/tracks/'
    appkey = 'fd2a2f47db6ed8f21988112c693c5d38'
    appid = '13f54a6b'
    maxflights = str(maxflights)
    url_req = url + air_code + '/arr?appId=' + appid + '&appKey=' + appkey + '%09&includeFlightPlan=false&maxPositions=1&maxFlights=' + maxflights
    
    response = requests.request("GET", url_req).json()
    
    for ubi in response['flightTracks']:
        # Extracción de datos
        id = str(ubi['flightId'])
        avion = ubi['callsign']
        lat = str(ubi['positions'][0]['lat'])
        long = str(ubi['positions'][0]['lon'])
        time = ubi['positions'][0]['date']
        velocidad = ubi['positions'][0]['speedMph']
        from_ = ubi['departureAirportFsCode']
        try:
            retraso = ubi['delayMinutes']
        except:
            retraso = 0
            
         # Ingreso a base de datos temporal
        if status((lat, long)) == 'A':
            r.set(id+':avion', avion)
            r.set(id+':lat', lat)
            r.set(id+':long', long)
            r.set(id+':time', time)
            r.set(id+':velocidad', velocidad)
            r.set(id+':from', from_)
            r.set(id+':retraso', retraso)
            
        if status((lat, long)) == 'L':
            r.delete(id+':avion', id+':lat', id+':long', id+':time', id+':velocidad', id+':from', id+':retraso')
            
            # Ingreso a base de datos histórica
            if len(list(vuelos.find({'_id':id}))) == 0:
                time = datetime.datetime.strptime(time[:-5], '%Y-%m-%dT%H:%M:%S')
                datos = {'_id': id, 'avion': avion, 'from': from_, 'retraso': retraso, 'time': time, 'ciudad':airports[from_]['city'],
                        'pais':airports[from_]['country']}
                vuelos.insert_one(datos)


In [141]:
# Iniciamos la simulación del flujo

def flujo(sleep, total):
    t = time.time()
    while True:
        ingresa('MEX', 100)
        time.sleep(sleep)
        if (time.time() - t) > total:
           break

In [275]:
flujo(30,120)

In [151]:
def promedio_retraso(tipo='Todo'):
    if tipo == 'Todo':
        return list(vuelos.aggregate([{'$group':{'_id':None, 'promedio':{'$avg':'$retraso'}}}]))[0]['promedio']
    if tipo == 'Aeropuerto':
        consulta = pd.DataFrame(list(vuelos.aggregate([{'$group':{'_id':'$from', 'promedio':{'$avg':'$retraso'}}}])))
        consulta.columns = ['Aeropuerto', 'Promedio']
        return consulta
    if tipo == 'Pais':
        consulta = pd.DataFrame(list(vuelos.aggregate([{'$group':{'_id':'$pais', 'promedio':{'$avg':'$retraso'}}}])))
        consulta.columns = ['Pais', 'Promedio']
        return consulta
    if tipo == 'Hora':  
        consulta = pd.DataFrame(list(vuelos.aggregate([{'$group':{'_id':{'$hour':'$time'}, 'promedio':{'$avg':'$retraso'}}}])))
        consulta.columns = ['Hora', 'Promedio']
        return consulta


In [ ]:
promedio_retraso('Todo')

In [259]:
promedio_retraso('Aeropuerto')

,Aeropuerto,Promedio
0,CZM,0.000000
1,ZLO,82.000000
2,YUL,2.000000
3,MAM,0.000000
4,GDL,22.000000
...,...,...
61,QRO,14.000000
62,VER,56.666667
63,VSA,8.666667
64,TIJ,18.500000


In [260]:
promedio_retraso('Pais')

,Pais,Promedio
0,MX,18.192771
1,LU,0.000000
2,PA,22.000000
3,CA,10.000000
4,ES,25.000000
5,HN,6.000000
6,DO,8.000000
7,US,25.760000
8,PE,21.666667
9,JP,13.000000


In [261]:
promedio_retraso('Hora')

,Hora,Promedio
0,18,19.969697
1,20,12.809524
2,21,21.952381
3,17,20.142857
4,16,8.666667
5,19,20.347826


In [267]:
def llegadas(tipo = 'Aeropuerto'):
    if tipo == 'Aeropuerto':
        consulta = pd.DataFrame(list(vuelos.aggregate([{'$group':{'_id':'$from', 'cuenta':{'$count':{}}}}])))
        consulta.columns = ['Aeropuerto', 'Cuenta']
        return consulta
    if tipo == 'Hora':
        consulta = pd.DataFrame(list(vuelos.aggregate([{'$group':{'_id':{'$hour':'$time'}, 'cuenta':{'$count':{}}}}])))
        consulta.columns = ['Hora', 'Cuenta']
        return consulta
    

In [262]:
llegadas('Aeropuerto')

,Aeropuerto,Cuenta
0,BJX,1
1,CTM,1
2,HUX,1
3,PTY,1
4,OAX,3
...,...,...
61,MZT,2
62,CEN,2
63,BOG,3
64,MAD,1


In [268]:
llegadas('Hora')

,Hora,Cuenta
0,21,21
1,17,28
2,18,33
3,19,23
4,20,21
5,16,6


In [208]:
def speed(limit = 10, rapidos=True):
    assert isinstance(rapidos, bool), 'Variable rapidos debe ser boolena'
    dicc = {}
    
    velo = '*velocidad*'
    for key in r.scan_iter(velo):
        dicc[key[:-10].decode("utf-8") ] = [r.get(key).decode("utf-8")]
        
    avion = '*avion*'
    for key in r.scan_iter(avion):
        dicc[key[:-6].decode("utf-8") ].append(r.get(key).decode("utf-8"))
        
    if rapidos:
        return sorted(list(dicc.values()), reverse=True)[:limit]
    else:
         return sorted(list(dicc.values()), reverse=False)[:limit]


In [270]:
speed(limit = 3, rapidos=False)

[['168', 'AAL2233'], ['168', 'SLI2469'], ['248', 'VIV1167']]

In [271]:
speed(limit = 5)

[['604', 'VOI749'],
 ['604', 'VOI701'],
 ['604', 'VIV281'],
 ['604', 'VIV1151'],
 ['604', 'AVA72']]

In [217]:
def lat_long():
    dicc = {}
    
    lat = '*lat*'
    for key in r.scan_iter(lat):
        dicc[key[:-4].decode("utf-8") ] = [r.get(key).decode("utf-8")]
        
    long = '*long*'
    for key in r.scan_iter(long):
        dicc[key[:-5].decode("utf-8") ].append(r.get(key).decode("utf-8"))
        
    avion = '*avion*'
    for key in r.scan_iter(avion):
        dicc[key[:-6].decode("utf-8") ].append(r.get(key).decode("utf-8"))
        
    return dicc


In [255]:
def grafica():
    # Recolección de coordenadas
    data = lat_long()
    
    # Inicialización el MAPA
    my_map = folium.Map(
        location = [19.436111, -99.071944],
        max_zoom = 7
    )

    folium.TileLayer('cartodbdark_matter').add_to(my_map)

    # Graficaciión del aeropuerto
    folium.CircleMarker(
            location = [19.436111, -99.071944],
            radius=10, color = 'red', popup = 'AICM', tooltip= 'AICM').add_to(my_map)
    
    # Graficación de vuelos
    for vuelo in data:
        folium.CircleMarker(
                location = [float(data[vuelo][0]), float(data[vuelo][1])],
                radius=1, color = 'orange', icon='plane', popup = data[vuelo][2], tooltip=data[vuelo][2]).add_to(my_map)
        
    # Muestra el mapa
    display(my_map)

In [273]:
grafica()